In [1]:
import math
import datetime
import pandas as pd
from scipy import optimize
from dateutil.relativedelta import relativedelta

# Calc Price, Duration, M_Duration, Conveixty

In [75]:
class calc_bond:
    
    @staticmethod
    def date(num):
        date = datetime.datetime.strptime(num, "%y/%m/%d").date()
        return date
    
    @staticmethod
    def truncate(number, digits) -> float:
        nbDecimals = len(str(number).split('.')[1]) 
        if nbDecimals <= digits:
            return number
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    
    # get required date, df
    def calc_df(self, issuedate, maturity, settlement, CR, YTM, period):
        
        # init
        K = int(12/period)
        schedule = self.date(issuedate)
        interval = relativedelta(months=period)
        
        i = 0
        after_cd = self.date(issuedate) + interval
        before_cd = self.date(issuedate)
        
        while schedule < self.date(maturity):
            
            # coupon num, schedule date
            i += 1
            schedule += interval
            
            # 직전이자 직후이자 구하기
            if self.date(settlement) >= schedule:
                before_cd = schedule   
                after_cd = schedule + interval 
            
            # DF of comp and simp(after_cd, before_cd 계산 이후 init)
            DF_comp_init = 1/(1+YTM/K)
            DF_simp_init = 1/(1+YTM/K*((after_cd - self.date(settlement))/(after_cd - before_cd)))    
            
        return after_cd, before_cd, DF_comp_init, DF_simp_init


    def get_price(self, issuedate, maturity, settlement, CR, YTM, period):

        after_cd, before_cd, DF_comp_init, DF_simp_init = self.calc_df(issuedate, maturity, settlement, CR, YTM, period)
       
        K = int(12/period)
        interval = relativedelta(months=period)
        schedule = after_cd - interval #  첫 스케줄 init
        CF = 10000*CR/K
        PV_CF = 0
        Price = 0
        i = 0
        df_CF = pd.DataFrame(columns = ['num','date','CF', 'DF', 'PV of CF'])
        
        
        while schedule < self.date(maturity):
            
            i += 1
            schedule += interval

            # 만기 + Principle ~ 규칙인 case만 cover中
            if schedule == self.date(maturity): 
                CF += 10000

            DF_comp = DF_comp_init ** (i-1)
            DF_simp = DF_simp_init
            DF = round(DF_comp*DF_simp, 4) # discount factor 소수점 4자리 까지
            PV_CF = CF*DF

            df_CF.loc[i] = [i, schedule, CF, DF, PV_CF] # dataframe CF 정리
            Price += PV_CF 

        df_CF.set_index(keys='num')
        print(f"[Price]: {Price}")
        print(f"YTM: {YTM}")
#         print(f"[Cash Flow]\n{df_CF}\n")
        return Price
    
    
    def calc_dur_conv(self, issuedate, maturity, settlement, CR, YTM, period):
        
        # init
        K = int(12/period)
        CF = 10000*CR/K
        PV_CF = 0
        interval = relativedelta(months=period)
        
        after_cd, before_cd, DF_comp_init, DF_simp_init = self.calc_df(issuedate, maturity, settlement, CR, YTM, period)
        schedule = after_cd - interval #  첫 스케줄 init
        dirty_day = (after_cd - self.date(settlement))/(after_cd - before_cd)
        price = self.get_price(issuedate, maturity, settlement, CR, YTM, period)
        
        i = 0
        tk = 0 # df of dur
        ttkk = 0 # df of conv
        duration = 0
        convexity = 0
        
        df_CF = pd.DataFrame(columns = ['num','date','CF', 'DF', 'W', 'tk',' ttkk'])
        
        while schedule < self.date(maturity):
            i += 1
            schedule += interval

            # 만기 + Principle
            if schedule == self.date(maturity): 
                CF += 10000

            DF_comp = DF_comp_init ** (i-1)
            DF_simp = DF_simp_init
            DF = self.truncate(DF_comp*DF_simp, 4) # 소수점 5자리 절사
            PV_CF = CF*DF
            weight = PV_CF/price
            
            if i == 1: # diry_day 기준
                tk = i/K*dirty_day             
                ttkk = dirty_day*(dirty_day+1)/(K**2)
        
            else:
                tk = (dirty_day + (i-1))/K 
                ttkk = ((dirty_day+i-1)*(dirty_day+i))/(K**2)   
        
            duration += weight*tk
            convexity += weight * ttkk * (1/(1+YTM/K)**2)
            df_CF.loc[i] = [i, schedule, CF,DF, weight, tk, ttkk] # dataframe CF 정리
        M_duration = duration/(1+YTM/K)
        
        
        print(f"[Duration]: {duration}, [M_Duration]: {M_duration},[Convexity]: {convexity}\n")
        print(f"[Cash Flow]\n {df_CF}")
        
        return duration, M_duration, convexity

### Ex1

In [76]:
calc1 = calc_bond()

# 입력값
issuedate = "14/03/10"
maturity = "19/03/10"
# settlement = "14/03/10"     # 기간
settlement = "15/02/23"     # 기간


CR = 0.03125
YTM = 0.022   # 금리
# YTM = 0.03125   # 금리

period = 6   # 주기

# calc1.calc_df(issuedate, maturity, settlement, CR, YTM, period)
# calc1.get_price(issuedate, maturity, settlement, CR, YTM, period)
dur, m_dur, conv = calc1.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)

[Price]: 10499.34375
YTM: 0.022
[Duration]: 3.7800997789933564, [M_Duration]: 3.738971096927158,[Convexity]: 16.49252768739909

[Cash Flow]
    num        date        CF      DF         W        tk       ttkk
1    1  2015-03-10    156.25  0.9990  0.014867  0.041436   0.022435
2    2  2015-09-10    156.25  0.9882  0.014706  0.541436   0.563872
3    3  2016-03-10    156.25  0.9774  0.014546  1.041436   1.605308
4    4  2016-09-10    156.25  0.9668  0.014388  1.541436   3.146745
5    5  2017-03-10    156.25  0.9563  0.014232  2.041436   5.188181
6    6  2017-09-10    156.25  0.9459  0.014077  2.541436   7.729618
7    7  2018-03-10    156.25  0.9356  0.013923  3.041436  10.771054
8    8  2018-09-10    156.25  0.9254  0.013772  3.541436  14.312490
9    9  2019-03-10  10156.25  0.9153  0.885390  4.041436  18.353927


### Ex2

In [77]:
calc2 = calc_bond()

# 입력값
issuedate = "22/12/19"
maturity = "24/12/19"
# settlement = "14/03/10"     # 기간
settlement = "22/12/19"     # 기간


# CR = 0.03125
CR = 0.042
# YTM = 0.022   # 금리
YTM = 0.04085   # 금리

period = 6   # 주기

# calc1.calc_df(issuedate, maturity, settlement, CR, YTM, period)
# calc2.get_price(issuedate, maturity, settlement, CR, YTM, period)
dur, m_dur, conv = calc1.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)
print(dur, m_dur, conv)

[Price]: 10021.798
YTM: 0.04085
[Duration]: 1.9392094113251934, [M_Duration]: 1.9003938665998907,[Convexity]: 4.607569736671393

[Cash Flow]
    num        date       CF      DF         W   tk   ttkk
1    1  2023-06-19    210.0  0.9799  0.020533  0.5    0.5
2    2  2023-12-19    210.0  0.9603  0.020122  1.0    1.5
3    3  2024-06-19    210.0  0.9411  0.019720  1.5    3.0
4    4  2024-12-19  10210.0  0.9223  0.939620  2.0    5.0
1.9392094113251934 1.9003938665998907 4.607569736671393


# Price -> YTM

In [80]:
# 입력값
issuedate = "14/03/10"
maturity = "19/03/10"
settlement = "14/03/10"     # 기간

CR = 0.03125
# YTM = 0.03125   # 금리

period = 6   # 주기

In [135]:
# bond_price = float(input("input price: "))
bond_price = 10530.39
print(f"[input price]: {bond_price}\n")

get_irr \
= lambda IRR: calc1.get_price(issuedate, maturity, settlement, CR, IRR, period) - bond_price

estimate = 0

IRR = optimize.newton(get_irr, estimate, disp=False)
print(f"\n[IRR]: {round(IRR, 5)}")

[input price]: 10530.39

[Price]: 11562.5
YTM: 0.0
[Price]: 11557.109375
YTM: 0.0001
[Price]: 10574.265625
YTM: 0.019146388405797236
[Price]: 10532.921875
YTM: 0.019996449665604273
[Price]: 10530.703125
YTM: 0.020048507083041598
[Price]: 10529.65625
YTM: 0.020055853777727835
[Price]: 10530.6875
YTM: 0.020050704512317906
[Price]: 10530.671875
YTM: 0.0200521899973695
[Price]: 10528.578125
YTM: 0.020078988147700302
[Price]: 10529.65625
YTM: 0.020055797748354344
[Price]: 10530.734375
YTM: 0.020040014833089335
[Price]: 10530.734375
YTM: 0.02004505621645805

[IRR]: 0.02004


C:\Users\jjgus\anaconda3\envs\infomax_project\lib\site-packages\scipy\optimize\_zeros_py.py:348: RuntimeWarning: Tolerance of 5.0413833687160126e-06 reached.
  warnings.warn(msg, RuntimeWarning)
